## The pre-processing 

In [105]:
import pandas as pd
import numpy as np

Links to data: https://factfinder.census.gov/faces/tableservices/jsf/pages/productview.xhtml?pid=ACS_pums_csv_2012_2016&prodType=document

Data dic: https://www2.census.gov/programs-surveys/acs/tech_docs/pums/data_dict/PUMS_Data_Dictionary_2012-2016.pdf

Please download "Rhode Island - Housing Unit Records" 

# 2012-2016 ACS PUMS DATA

In [107]:
rhode_island = pd.read_csv("clean_data.csv")

In [108]:
rhode_island.head()

,RT,SERIALNO,DIVISION,PUMA,REGION,ST,ADJHSG,ADJINC,WGTP,NP,...,WGTP71,WGTP72,WGTP73,WGTP74,WGTP75,WGTP76,WGTP77,WGTP78,WGTP79,WGTP80
0,H,2012000000026,1,201,1,44,1045360,1056030,0,1,...,0,0,0,0,0,0,0,0,0,0
1,H,2012000000447,1,300,1,44,1045360,1056030,0,1,...,0,0,0,0,0,0,0,0,0,0
2,H,2012000000506,1,103,1,44,1045360,1056030,20,1,...,20,30,22,15,7,5,29,31,6,33
3,H,2012000000658,1,300,1,44,1045360,1056030,10,2,...,17,17,10,10,9,2,11,16,10,9
4,H,2012000001192,1,101,1,44,1045360,1056030,23,2,...,21,7,22,7,42,39,22,27,25,40


In [79]:
rhode_island.shape

(26825, 208)

In [44]:
#california = pd.read_csv("csv_hca/ss16hca.csv")
#california.head()

,RT,SERIALNO,DIVISION,PUMA,REGION,ST,ADJHSG,ADJINC,WGTP,NP,...,WGTP71,WGTP72,WGTP73,WGTP74,WGTP75,WGTP76,WGTP77,WGTP78,WGTP79,WGTP80
0,H,2012000000011,9,8508,4,6,1045360,1056030,21,2,...,37,39,23,22,34,5,24,7,7,22
1,H,2012000000019,9,7502,4,6,1045360,1056030,34,0,...,56,37,57,9,36,55,51,37,36,40
2,H,2012000000022,9,8105,4,6,1045360,1056030,14,1,...,15,25,23,22,14,4,14,24,14,25
3,H,2012000000031,9,1305,4,6,1045360,1056030,19,2,...,20,6,21,21,36,38,6,7,30,30
4,H,2012000000040,9,2901,4,6,1045360,1056030,2,2,...,3,5,4,4,2,2,2,2,2,1


In [45]:
#california.shape

(772328, 208)

We now remove the columns with information that won't be necessary for rent prediction

In [109]:
index = np.where(rhode_island.columns == 'WORKSTAT')[0][0]
rhode_island = rhode_island[rhode_island.columns[:index+1]]
rhode_island.drop(['AGS','RT', 'DIVISION', 'REGION', 'ST'],axis=1, inplace=True)
print(rhode_island.columns.unique())

Index(['SERIALNO', 'PUMA', 'ADJHSG', 'ADJINC', 'WGTP', 'NP', 'TYPE', 'ACR',
       'BATH', 'BDSP', 'BLD', 'BUS', 'CONP', 'ELEP', 'FS', 'FULP', 'GASP',
       'HFL', 'INSP', 'MHP', 'MRGI', 'MRGP', 'MRGT', 'MRGX', 'REFR', 'RMSP',
       'RNTM', 'RNTP', 'RWAT', 'RWATPR', 'SINK', 'SMP', 'STOV', 'TEL', 'TEN',
       'TOIL', 'VACS', 'VALP', 'VEH', 'WATP', 'YBL', 'FES', 'FINCP', 'FPARC',
       'GRNTP', 'GRPIP', 'HHL', 'HHT', 'HINCP', 'HUGCL', 'HUPAC', 'HUPAOC',
       'HUPARC', 'KIT', 'LNGI', 'MULTG', 'MV', 'NOC', 'NPF', 'NPP', 'NR',
       'NRC', 'OCPIP', 'PARTNER', 'PLM', 'PSF', 'R18', 'R60', 'R65', 'RESMODE',
       'SMOCP', 'SMX', 'SRNT', 'SVAL', 'TAXP', 'WIF', 'WKEXREL', 'WORKSTAT'],
      dtype='object')


In [110]:
rhode_island.shape

(26825, 78)

We exclude institutional and non-institutional units and only leave housing units in our model. Ref: https://www.census.gov/topics/income-poverty/poverty/guidance/group-quarters.html. Then, the dataset size narrows to following:

In [111]:
rhode_island = rhode_island[rhode_island.TYPE == 1]
rhode_island.shape

(23122, 78)

We remove non-common building types, such as boats, vans and mobile homes as they're technically not "real" estate.

In [112]:
rhode_island = rhode_island[(rhode_island.BLD > 1) & (rhode_island.BLD <= 9)]
rhode_island.shape

(22894, 78)

The question about business or medical office on property was dropped in 2016 in an effort to reduce respondent burden. We omit this data in our dataset for consisent prediction and also due to the fact we predict housing rent and not business rent.

In [113]:
rhode_island = rhode_island[rhode_island.BUS == 2]
rhode_island.shape

(11598, 78)

In [119]:
rhode_island.SMX = rhode_island.SMX.fillna(-1)

In [120]:
rhode_island.isnull().sum()

SERIALNO        0
PUMA            0
ADJHSG          0
ADJINC          0
WGTP            0
NP              0
TYPE            0
ACR             0
BATH            0
BDSP            0
BLD             0
BUS             0
CONP         1880
ELEP          743
FS            743
FULP          743
GASP          743
HFL           743
INSP         1880
MHP         11598
MRGI         4971
MRGP         4971
MRGT         4971
MRGX         1880
REFR            0
RMSP            0
RNTM        10561
RNTP        10561
RWAT            0
RWATPR          0
            ...  
HINCP         743
HUGCL         743
HUPAC         743
HUPAOC        743
HUPARC        743
KIT             0
LNGI          743
MULTG         743
MV            743
NOC           743
NPF          3561
NPP           743
NR            743
NRC           743
OCPIP        1935
PARTNER       743
PLM             0
PSF           743
R18           743
R60           743
R65           743
RESMODE         0
SMOCP        1880
SMX             0
SRNT      

In [114]:
rhode_island.to_csv("clean_data.csv",header=True)